This tutorial will guide you through the process of visualizing Landsat data that covers your country by overlaying Landsat scenes on your country's borders. For example, we will use Egypt. We will use the false color band composition of Landsat-8 OLI, where the "green area" appears in red. Please make sure to adjust the geographic coordinates [Lat, Long] on line #9 to match your country's specifications. For Egypt, the coordinates should be adjusted to [31,32]. Also, remember to update the country name on line #4 (countries filter) and line #5 (filter bounds). Before beginning, you must install the Conda package and the Python package, specifically the folium library.

Best regards,
Dr. Basam Dahy (dohaibassam@gmail.com)

In [1]:
import ee

In [2]:
ee.Authenticate()

Enter verification code: 4/1AWtgzh5-m12S92I3FIMxYb0FkHjONY8PiN4rTDQTudX9CQK1d4VLEuNEruA

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
Egypt = countries.filter(ee.Filter.eq("country_na", "Egypt"))

In [5]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1").filterDate("2016-01-01", "2017-01-01").filterBounds(Egypt)
composite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': landsat,
    'asFloat': True
})

rgbVis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
nirVis = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': [0.5, 0.3, 0.3]}

In [6]:
import folium

In [7]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

In [8]:
folium.Map.add_ee_layer = add_ee_layer

In [9]:
my_map = folium.Map(location=[31, 32], zoom_start = 6)

In [10]:
my_map.add_ee_layer(composite, rgbVis, "RGB")
my_map.add_ee_layer(composite, nirVis, "False Color")

In [11]:
my_map.add_child(folium.LayerControl())
display(my_map)